# MODULES

In [18]:
!pip install pathlib
!pip install matplotlib
!pip install ultralytics
!pip install opencv-contrib-python
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu117
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 GB 3.7 MB/s eta 0:00:00m eta 0:00:010:00:08
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 5.7 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 5.1 MB/s eta 0:00:00 MB/s eta 0:00:01:01
  Attempting uninstall: torch
    Found existing installation: torch 2.8.0
    Uninstalling torch-2.8.0:━━━━━━━━━━━━━━━━━━━━━━━━━ 0/3 [torch]
      Successfully uninstalled torch-2.8.0━━━━━━━━━━━━ 0/3 [torch]
  Attempting uninstall: torchvision━━━━━━━━━━━━━━━━━━━ 0/3 [torch]
    Found existing installation: torchvision 0.23.0 0/3 [torch]
    Uninstalling torchvision-0.23.0:━━━━━━━━━━━━━━ 0/3 [torch]
      Successfully uninstalled torchvision-0.23.0━ 0/3 [torch]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [torchaudio]━━━━━━━ 2/3 [torchaudio]


In [19]:
!nvidia-smi

Mon Aug 25 14:21:50 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.169                Driver Version: 570.169        CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce MX330           Off |   00000000:2C:00.0 Off |                  N/A |
| N/A   51C    P8            N/A  / 5001W |     251MiB /   2048MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.get_device_name(0))

2.8.0+cu128
12.8
NVIDIA GeForce MX330


In [21]:
import cv2
import math
import random
import pathlib
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt

# GLOBAL VARIABLES

In [22]:
ROOT_DIR_IMAGES = './Kaggle/archive'

# VIDEOS

In [23]:
def get_videos(root_dir):
  paths = []
  for path in pathlib.Path(root_dir).glob('*.mp4'):
    paths.append(path)

  paths = sorted(paths)

  return paths

videos = {}
videos['fall']     = get_videos( ROOT_DIR_IMAGES + '/Fall/Raw_Video')
videos['no_fall'] = get_videos( ROOT_DIR_IMAGES + '/No_Fall/Raw_Video')

print(f"[INFO] videos dict keys: {list(videos.keys())}")
print(f"[INFO] videos['fall'] has {len(videos['fall'])} entries")
print(f"[INFO] videos['no_fall'] has {len(videos['no_fall'])} entries")

[INFO] videos dict keys: ['fall', 'no_fall']
[INFO] videos['fall'] has 3140 entries
[INFO] videos['no_fall'] has 3848 entries


In [24]:
def get_person_crops(video_path, model_path="yolov8l.pt", conf_thres=0.5):
    """
    Reads a video, applies YOLO person detection, and returns cropped person images.

    Args:
        video_path (str): Path to the .mp4 video
        model_path (str): Path to YOLO model (default = yolov8n.pt)
        conf_thres (float): Confidence threshold for detection

    Returns:
        List of cropped person images (numpy arrays in BGR)
    """
    # Load YOLO
    model = YOLO(model_path)

    crops = []
    cap = cv2.VideoCapture(str(video_path))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Run YOLO detection on frame
        results = model(frame, conf=conf_thres, verbose=False)

        # Extract person detections (class 0 in COCO)
        for box in results[0].boxes:
            cls = int(box.cls[0])
            if cls == 0:  # person
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                crop = frame[y1:y2, x1:x2]
                if crop.size > 0:
                    crops.append(crop)

    cap.release()
    return crops

def resize_images(imgs, size=(300,300)):
  lst = []
  for img in imgs:
    lst.append(cv2.resize(img, size))
  return lst


def sub_sample(lst, elens=25):
    """
    Subsample a list to have at most `elens` elements, evenly spaced.
    Indices are generated first to avoid repetition.

    Args:
        lst (list): Input list
        elens (int): Desired number of elements

    Returns:
        list: Subsampled list
    """
    n = len(lst)
    if n <= elens:
        return lst

    step = n / elens
    indices = [int(i * step) for i in range(elens)]
    indices = list(dict.fromkeys(indices))  # remove any accidental repetitions

    return [lst[i] for i in indices], indices  

In [25]:
path = videos['fall'][0]
print(path)
crops = get_person_crops(path, model_path='yolo11l.pt')

Kaggle/archive/Fall/Raw_Video/20240912_101331.mp4


AcceleratorError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
